# Process scheduling

A set of n jobs must be processed in a machine that can handle one job at a time.
Task j needs pj hours to be processed.
A directed and acyclic graph G = (V, E), with V = {1, . . . , n}, establishes a partial
order for job processing in the machine. That is, if there exists a path δi,j from i to
j in G, then job i must be processed before job j.
Given nonnegative weights wj , j = 1, . . . , n, in which order should we process the
jobs in order to minimize the weighted sum of the start processing time of all jobs,
while respecting the precedence order? For the modeling task that follows, sj is the
instant that job j starts to be processed.
Tasks:<br>
(a) Formulate the problem in mixed-integer linear programming using discrete and
continuous variables.<br>
(b) Model the problem in AMPL and solve the instance given below, in which
V = {1, . . . , 12}. Present the results. <br>

| job (j)  | length (pj)| weight (wj) | Arcs (j,i)    |
|----|----|----|---------------|
| 1  | 3  | 5  | (1,3)         |
| 2  | 2  | 3  |               |
| 3  | 6  | 7  | (3,12), (3,7) |
| 4  | 2  | 6  |               |
| 5  | 5  | 1  |               |
| 6  | 4  | 2  | (6,7)         |
| 7  | 4  | 8  |               |
| 8  | 3  | 4  | (8,6)         |
| 9  | 10 | 7  |               |
| 10 | 1  | 1  | (10,12)       |
| 11 | 8  | 6  |               |
| 12 | 7  | 2  |               |

# Model

\begin{equation}
\min  \sum_{j =1}^{N} s_j \times w_j \\
\mathrm{s.t.} : N \in \mathbb{Z}^{+} \\
s_{j} \ge (s_i+p_i).b_{i,j} \\
s_{i} \ge (s_j+p_j).\hat{b}_{i,j} \\
s_j \in \mathbb{R}_{\ge 0} \space \forall j \\
p_j \in \mathbb{R}_{+} \\
w_j \in \mathbb{R}_{+} \\
b_{i,j} \in \{0,1\} \space \forall i,j \\
\hat{b}_{i,j} \in \{0,1\} \space \forall i,j \\
b_{i,j}=1-\hat{b}_{i,j}\space \forall i,j \\
b_{j,j}=0\space \forall j \\
arc_{i,j} \in \{0,1\} \space \forall i,j \\
b_{i,j} \ge arc_{i,j}
\end{equation}

## Modeling (instance)

In [1]:
import gurobipy as gp
from gurobipy import GRB, Model
import pandas as pd
import numpy as np

### Create supporting (instance) data

In [2]:
N = 12; # 'size' of the problem, here it's both the number of processes and number of slots

# Create a list with the well indexes as given by the problem (prevents python's 0-indexing)
V = [item for item in range(1, N+1)]

# supporting data:
w = {j:[5,3,7,6,1,2,8,4,7,1,6,2][j-1] for j in V} # these are the given process 'weights' (importances)
p = {j:[3,2,6,2,5,4,4,3,10,1,8,7][j-1] for j in V} # these are the given process time lenghts

"""
'dependance table'
     p1 p2 p3 p4
p1  [0  1  0  0] <- p1 must come before p2
p2  [0  0  0  0]
p3  [1  0  0  0] <- p3 must come before p1
p4  [0  0  0  0]
"""
D = np.zeros((N,N))
arcs = ((1,3),(3,12),(3,7),(6,7),(8,6),(10,12)) # these arcs are in order (i,j), where i depends on j
for arc in arcs:
    x=int(arc[0]) - 1 # minus one is because of python 0 indexing
    y=int(arc[1]) - 1 # minus one is because of python 0 indexing
    D[x][y] = 1
print('dependance table\n',D)

dependance table
 [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [3]:
# model creation
m = Model("process_scheduling") 

# Decision variables:
# 1)represents the starting time of each process
'''
s: [1, 3, 4, ...] process 1 starts at time=1, process 2 starts at time=3...
'''
s = m.addVars(V, vtype=GRB.CONTINUOUS, name="s") 

# 2)decision varibles. If B[i,j]=1, then process i comes before process j
"""
'comes before table'
     p1 p2 p3 p4
p1  [0  1  1  1] p1 comes before p2, p3 and p4 - first
p2  [0  0  0  0] p2 comes before no one-last
p3  [0  1  0  1] p3 comes before p2 and p4 - second
p4  [0  1  0  0] p4 - third
p1->p3->p4->p2
"""

B=m.addVars(V, V, vtype=GRB.BINARY, name='B')
NB=m.addVars(V, V, vtype=GRB.BINARY, name='NB') # negation of B

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [4]:
#Constraints over the binary tables
for i in V:
    for j in V:
        m.addConstr(B[i,j]==1-NB[i,j], name=f"BNB{i}_{j}") # if job i comes before job j, then job j cannot come before job i

m.update()

In [5]:
# Constraints related to the
# 1)non-overlaping usages of resources
for j in V:
    for i in (set(V)-set([j])):
        m.addConstr(s[j]>=(s[i]+p[i])*B[i,j])
        m.addConstr(s[i]>=(s[j]+p[j])*NB[i,j])

# 2)given process precedence requirements 
for i in V:
    for j in V:
        Di = i - 1
        Dj = j - 1
        if D[Di][Dj] == 1:
            m.addConstr(B[i,j]==1,name=f'arc({i},{j})')

# 3) a job cannot happen before itself            
for i in V:
    m.addConstr(B[i,i]==0)

m.update()

In [6]:
m.setObjective(sum([s[j]*w[j] for j in V]), GRB.MINIMIZE)

m.update()
m.display()

Minimize
<gurobi.LinExpr: 5.0 s[1] + 3.0 s[2] + 7.0 s[3] + 6.0 s[4] + s[5] + 2.0 s[6]
+ 8.0 s[7] + 4.0 s[8] + 7.0 s[9] + s[10] + 6.0 s[11] + 2.0 s[12]>
Subject To
  BNB1_1: <gurobi.LinExpr: B[1,1] + NB[1,1]> = 1
  BNB1_2: <gurobi.LinExpr: B[1,2] + NB[1,2]> = 1
  BNB1_3: <gurobi.LinExpr: B[1,3] + NB[1,3]> = 1
  BNB1_4: <gurobi.LinExpr: B[1,4] + NB[1,4]> = 1
  BNB1_5: <gurobi.LinExpr: B[1,5] + NB[1,5]> = 1
  BNB1_6: <gurobi.LinExpr: B[1,6] + NB[1,6]> = 1
  BNB1_7: <gurobi.LinExpr: B[1,7] + NB[1,7]> = 1
  BNB1_8: <gurobi.LinExpr: B[1,8] + NB[1,8]> = 1
  BNB1_9: <gurobi.LinExpr: B[1,9] + NB[1,9]> = 1
  BNB1_10: <gurobi.LinExpr: B[1,10] + NB[1,10]> = 1
  BNB1_11: <gurobi.LinExpr: B[1,11] + NB[1,11]> = 1
  BNB1_12: <gurobi.LinExpr: B[1,12] + NB[1,12]> = 1
  BNB2_1: <gurobi.LinExpr: B[2,1] + NB[2,1]> = 1
  BNB2_2: <gurobi.LinExpr: B[2,2] + NB[2,2]> = 1
  BNB2_3: <gurobi.LinExpr: B[2,3] + NB[2,3]> = 1
  BNB2_4: <gurobi.LinExpr: B[2,4] + NB[2,4]> = 1
  BNB2_5: <gurobi.LinExpr: B[2,5] + NB[2,5]>

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 162 rows, 300 columns and 306 nonzeros
Model fingerprint: 0x7ca0adbe
Model has 264 quadratic constraints
Variable types: 12 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 156 rows and 162 columns
Presolve time: 0.01s
Presolved: 762 rows, 894 columns, 2154 nonzeros
Presolved model has 504 SOS constraint(s)
Variable types: 516 continuous, 378 integer (378 binary)

Root relaxation: objective 2.478783e+02, 506 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [8]:
m.getVars()

[<gurobi.Var s[1] (value 2.0000000001994516)>,
 <gurobi.Var s[2] (value 5.000000000467336)>,
 <gurobi.Var s[3] (value 10.000000001192044)>,
 <gurobi.Var s[4] (value 3.468917672391373e-11)>,
 <gurobi.Var s[5] (value 50.00000004067036)>,
 <gurobi.Var s[6] (value 16.00000000166354)>,
 <gurobi.Var s[7] (value 20.000000002230056)>,
 <gurobi.Var s[8] (value 7.000000000847653)>,
 <gurobi.Var s[9] (value 33.00000000458675)>,
 <gurobi.Var s[10] (value 24.000000002939192)>,
 <gurobi.Var s[11] (value 25.0000000035599)>,
 <gurobi.Var s[12] (value 43.00000000786127)>,
 <gurobi.Var B[1,1] (value 0.0)>,
 <gurobi.Var B[1,2] (value 1.0)>,
 <gurobi.Var B[1,3] (value 1.0)>,
 <gurobi.Var B[1,4] (value 0.0)>,
 <gurobi.Var B[1,5] (value 1.0)>,
 <gurobi.Var B[1,6] (value 1.0)>,
 <gurobi.Var B[1,7] (value 1.0)>,
 <gurobi.Var B[1,8] (value 1.0)>,
 <gurobi.Var B[1,9] (value 1.0)>,
 <gurobi.Var B[1,10] (value 1.0)>,
 <gurobi.Var B[1,11] (value 1.0)>,
 <gurobi.Var B[1,12] (value 1.0)>,
 <gurobi.Var B[2,1] (value 

In [9]:
for j in V:
    start=round(s[j].x)
    end = start+p[j]
    print(f'Process {j} starts at {start} and ends at {end}')

Process 1 starts at 2 and ends at 5
Process 2 starts at 5 and ends at 7
Process 3 starts at 10 and ends at 16
Process 4 starts at 0 and ends at 2
Process 5 starts at 50 and ends at 55
Process 6 starts at 16 and ends at 20
Process 7 starts at 20 and ends at 24
Process 8 starts at 7 and ends at 10
Process 9 starts at 33 and ends at 43
Process 10 starts at 24 and ends at 25
Process 11 starts at 25 and ends at 33
Process 12 starts at 43 and ends at 50
